In [ ]:
# colab
from google.colab import drive
drive.mount('/content/drive')

# Unet - Chest CT Dataset

## 데이터 전처리

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

import os
import numpy as np
import pandas as pd
import shutil
import cv2
import matplotlib.pyplot as plt

from IPython import display
from PIL import Image

### kaggle datasets Download
- 데이터 : Chest CT Segmentation(Chest CT scans together with segmentation masks for lung, heart, and trachea)
- 캐글 데이터 주소: https://www.kaggle.com/datasets/polomarco/chest-ct-segmentation

In [ ]:
!pip install kaggle --upgrade

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d polomarco/chest-ct-segmentation

In [ ]:
!unzip -qq '/content/chestct/chest-ct-segmentation.zip'
%cd ..

In [ ]:
data_raw = pd.read_csv('/content/chestct/train.csv')
data_raw.head()

In [ ]:
data_raw.shape

In [ ]:
def get_id(x):
    return x.split('_')[0]

data_raw['id'] = data_raw.ImageId.apply(lambda x: get_id(x))
data_raw.head()

In [ ]:
cli_ids = data_raw.id.unique()
len(cli_ids)

In [ ]:
print(len(os.listdir('/content/chestct/images/images')), len(os.listdir('/content/chestct/masks/masks')))

In [ ]:
cli_id = -1
cli_data = data_raw[data_raw.id == cli_ids[cli_id]]
cli_data

In [ ]:
def get_cli_data(data_raw, cli_id):
    cli_ids = data_raw.id.unique()
    cli_data = data_raw[data_raw.id == cli_ids[cli_id]]
    
    image_file = cli_data.imageId.values
    mask_file = cli_data.Maskid.values
    id_file = cli_data.id.values[0]

    return id_file, image_file, mask_file

In [ ]:
idx = 0
get_cli_data(data_raw, idx)